# La théorie de l'inférence de causes de Judea Pearl

Judea Pearl a obtenu le prix Turing en 2011 principalement pour deux travaux.
Un algorithme de résolution des réseaux bayesiens dont nous ne discutons pas dans ce cours.
Une théorie de l'inférence de cause qui est l'objet de la dernière partie de ce cours.
Les plus motivés peuvent aller lire une présentation de ses travaux ici: https://amturing.acm.org/award_winners/pearl_2658896.cfm

Plan des séances à venir:

+ Notion de probabilité conditionnelle $P(X,Y|Z)$ et d'indépendance conditionnelle $P(X \perp\!\!\!\perp Y | Z)$.

+ Exemples de modélisation et hypothèses statistiques en résultant (Modèles graphiques probabilistes). 

+ Modèle (probabiliste) de causes comme des programmes à affectation unique des variables (en amont de la théorie).

+ Représentation par un graphe (variantes Markovienne et semi-Markovienne) des hypothèses statistiques.

+ Projection semi-Markovienne de tout graphe de cause [Verma 1993].

+ L'ensemble des indépendances conditionnelles d'un modèle de cause.

+ Terminologie en théorie des graphes.

+ Digression: Graphoids et séparation des ensembles de sommets $X$ et $Y$ par l'ensemble $Z$ comme autre incarnation des axiomes.

+ $d$-séparation de $X$ et $Y$ par $Z$ dans le graphe de cause impliquant l'indépendance conditionnelle $P(X \perp\!\!\!\perp Y | Z)$ 

+ Notion d'intervention, l'opérateur __do__ comme dans $P(...|do(X=x))$ et la différence avec $P(...|X=x)$.

+ Les trois barreaux de l'échelle de Pearl: __Observer__ (avec $P(...|X)$), __Expérimenter__ (avec $P(...|do(X=x)$), __Imaginer__ (hors de ce cours). 

+ Exemples de réduction d'une intervention à des observations: __back-door criterion__ de [Pearl 1993].

+ Les trois règles du __do-calculus__ de [ Pearl 2000] autorisées par des indépendances conditionnelles.

+ Exemples de __haies__ (hedges), les obstacles à l'observation des interventions.

+ Complétude du do-calculus: Les trois régles et les calculs de probabilité suffisent à calculer les interventions $P(...|do(X=x))$ lorsque c'est possible.

+ Algorithme de [Shpister et Pearl 2006]  pour décider si et si oui comment calculer une intervention $P(...|do(X=x))$ à partir d'observations.

+ Éléments de preuve de l'algorithme de Shpister et Pearl

+ De la preuve d'observabilité aux estimateurs statistiques (en aval de la théorie).

## Modéle de causes et leurs graphes de causes

Principalement au tableau.

- Probabilités
- Variables aléatoires indépendantes
- indépendance conditionnelle $X \perp \!\!\! \perp Y | Z$ sur un premier exemple.
- Reformulation de cette indépendance par un modèle de cause
- Modèles de cause et le graphe associé
- Énoncé de la projection de Verma sur les modèles de cause semi-markovien introduisant les arêtes bi-dirigées. 
- Exemple de modélisation
- Critère d'indépendances conditionnelles sur le graphe de cause ($d$-séparabilité). 

TODO 0.1: Nous formalisons (au tableau) l'exemple 3.19 du rapport technique "Algorithmique causal effect identification with causaleffect" de Pedemonte, Vitria, Parafita:    https://arxiv.org/abs/2107.04632

Example 3.19. Suppose we want to know how the salary of an employee in a given company depends on
the gender of the worker. It is reasonable to suppose that salary (Y) could depend on the level of education
(E) of the employee (better academic records usually translate to higher payroll), the field the employee is
working in (F) and the amount of time he has been working for the company (S for seniority). Clearly, the
age (A) of the worker influences both the level of education and the seniority, and gender (X) may have an
impact on the seniority as well as the field of the employee. In addition, we might think that both age and
gender may be related through a third unobserved variable (U), which could be that perhaps the company
used to hire only men in the past, and thus the average age of men is higher than that of women in the
company.

## Probabilité conditionnelle $P(X,Y|Z)$.

Ici $X$ dénote un ensemble de variables aléatoires $X=\{X_0,\ldots X_{n_X-1}\}$, ainsi que $Y=\{Y_0,\ldots,Y_{n_Y-1}\}$ et $Z=\{Z_0,\ldots Z_{n_Z-1}\}$.
Mais pour le début de la discussion vous pouvez supposer que $n_X=1=n_Y=n_Z$.



Dans une analogie avec le langage python, supposons que l'ensemble des observations soit $\{(X(\omega),Y(\omega),Z(\omega))\}_{\omega \in \Omega}$ où $\Omega$ est le maintenant si familier espace de probabilité (supposé dans ce cours discret et fini) munissant chaque événement $\omega$ d'une probabilité $p(\omega)$.

### Variables "jointes" et "disjointes"

TODO 1.1: Réviser la notion d'espace de probabilité sur la classe Python proposée. Complétez la fonction __join__ qui calcule à partir de la liste de variables $(X_1,\ldots X_n)$, $(Y_1,\ldots Y_k)$ et $(Z_1,\ldots Z_m)$, la variable "jointe" $(X_1,\ldots X_n,Y_1,\ldots Y_k,Z_1,\ldots Z_m)$. De même pour la fonction __split__ qui étant donné $(X_1,\ldots X_n)$ et un sous-ensemble $I$ de ses indices sépare les variables jointes en deux ensembles de variables jointe: les $(X_i)_{i\in I}$ et les autres $(X_i)_{i\notin I}$.


In [16]:
import copy
from typing import *
import fractions
import itertools

class ProbabilitySpace:
    
    def __init__(self,nb_events=None,measure:Dict=None):
        assert(nb_events != None or measure != None)
        if measure == None: # uniform measure on events by default
            self.measure = {omega: fractions.Fraction(1,nb_events) for omega in range(nb_events)}
        else:
            self.measure = measure
            
    def all_omegas(self):
        return self.measure.keys()

In [17]:
d12 = ProbabilitySpace(nb_events=12)
for omega in d12.all_omegas():
    print(omega,d12.measure[omega])

0 1/12
1 1/12
2 1/12
3 1/12
4 1/12
5 1/12
6 1/12
7 1/12
8 1/12
9 1/12
10 1/12
11 1/12


Nous nous donnons la possibilité de nommer une fonction anonyme au moment de sa déclaration.

In [18]:
def name(name:str,function:Callable)->Callable:
    function.__name__ = name
    function.__qualname__ = name
    return function

In [19]:
toto = name('X',lambda omega: omega%2)
print(toto)

<function X at 0x00000185B811CA60>


Nous adoptons la convention de __refuser__ les variables aléatoires qui ne sont pas nommées.

In [22]:
class RandomVariables:
    
    def __init__(self,Omega:ProbabilitySpace,Xs:Tuple[Callable]=None,Ys:Tuple[Callable]=None):
        ''' 
            .Xs: (may be) conditionned variables
            .Ys: (may be) conditionning variables
            .Zs: (may be) hidden variables
            '''
        if type(Xs) == tuple:
            self.Xs = Xs
        elif Xs == None:
            self.Xs = tuple([])
        else:
            self.Xs = (Xs,)
        if type(Ys) == tuple:
            self.Ys = Ys
        elif Ys == None:
            self.Ys = tuple([])
        else:
            self.Ys = (Ys,)
        self.Zs = tuple([])
        for i in range(len(self.Xs)):
            #print(f'{i}: {self.Xs[i].__qualname__}')
            assert( self.Xs[i].__qualname__ != '<lambda>')
        self.Omega = Omega
        
    def eval_at_event(self,omega):
        #print(f'eval_at_event self {self} omega {omega}')
        return tuple([ self.Xs[i](omega) for i in range(len(self.Xs)) ])
        
    def distribution(self):
        # Remark: it corresponds to conditionning by all free variables !
        assert(len(self.Ys) == 0) # Make sens only if no conditionning variables
        d = {}
        for omega in self.Omega.all_omegas():
            #print(f'omega {omega}')
            Xomega = self.eval_at_event(omega)
            if Xomega not in d:
                d[Xomega] = 0
            d[Xomega] += self.Omega.measure[omega]
        return d
    
    def distributions(self)->Dict:
        # Return distributions indexed by evaluation of conditionning variables:
        # - probabilities of evaluation
        # - random variable of a part of the original probability space (with change of measure)
        Omega_partition = {}
        for omega in self.Omega.all_omegas():
            ys = tuple([ self.Ys[j](omega) for j in range(len(self.Ys))])
            #print(f'omega {omega} ys {ys}')
            if ys not in Omega_partition:
                Omega_partition[ys] = {}
            Omega_partition[ys][omega] = self.Omega.measure[omega]
        #print(f'Omega_partition {Omega_partition}')
        ys_probabilities = { ys: sum(p for p in Omega_partition[ys].values()) for ys in Omega_partition}
        #print(f'ys_probabilities {ys_probabilities}')
        ys_probabilities = { ys: ys_probabilities[ys] for ys in ys_probabilities if ys_probabilities[ys] > 0}
        Omega_partition = { ys: {omega : Omega_partition[ys][omega]/ys_probabilities[ys]\
                                 for omega in Omega_partition[ys]} for ys in ys_probabilities}
        ys_conditional_variables = {ys: RandomVariables(ProbabilitySpace(measure=Omega_partition[ys]),Xs=self.Xs)\
                                   for ys in ys_probabilities}
        return (ys_conditional_variables,ys_probabilities)
       
    def move_by_name_from_to(self,names,from_Zs,to_Zs):
        for a_name in names:
            for Z in from_Zs:
                if Z.__qualname__ == a_name:
                    from_Zs.remove(Z)
                    to_Zs.insert(0,Z)
        return (from_Zs,to_Zs)
        
    def condition(self,names_new_conditionning_variables:List[str]):
        (new_Xs,new_Ys) = self.move_by_name_from_to(names_new_conditionning_variables,list(self.Xs),list(self.Ys))
        self.Xs = tuple(new_Xs)
        self.Ys = tuple(new_Ys)
        #for a_name in names_new_conditionning_variables:
        #    for i in range(len(self.Xs)):
        #        if self.Xs[i].__qualname__ == a_name:
        #            self.Ys = tuple([self.Xs.pop(i)]+list(self.Ys))
        
    def uncondition(self,names_new_conditionned_variables:List[str]):
        (new_Ys,new_Xs) = self.move_by_name_from_to(names_new_conditionned_variables,list(self.Ys),list(self.Xs))
        self.Xs = tuple(new_Xs)
        self.Ys = tuple(new_Ys)
        
    def hide(self,names_hidden_variables:List[str]):
        (new_Xs,new_Zs) = self.move_by_name_from_to(names_hidden_variables,list(self.Xs),list(self.Zs))
        self.Xs = tuple(new_Xs)
        self.Zs = tuple(new_Zs)
        
    def show(self,name_shown_variables:List[str]):
        (new_Zs,new_Xs) = self.move_by_name_from_to(name_shown_variables,list(self.Zs),list(self.Xs))
        self.Zs = tuple(new_Zs)
        self.Xs = tuple(new_Xs)
        
    def are_independant(self,name_Xs:List[str],name_Ys:List[str],print_counter_example=False):
        Xs = [ X for X in list(self.Xs+self.Ys+self.Zs) if X.__qualname__ in name_Xs ]
        Ys = [ Y for Y in list(self.Xs+self.Ys+self.Zs) if Y.__qualname__ in name_Ys ]
        marginal_X = RandomVariables(self.Omega,tuple(Xs)).distribution()
        marginal_Y = RandomVariables(self.Omega,tuple(Ys)).distribution()
        joint_XY = RandomVariables(self.Omega,tuple(Xs+Ys)).distribution()
        for x in marginal_X:
            for y in marginal_Y:
                if joint_XY[x+y] != marginal_X[x]*marginal_Y[y]:
                    if print_counter_example:
                        print(f'join({x+y}) = {joint_XY[x+y]},'+\
                        f'marginal_X[{x}]={marginal_X[x]}, marginal_Y[{y}]={marginal_Y[y]},'+\
                        f'marginal_X[{x}]*marginal_Y[{y}]={marginal_X[x]*marginal_Y[y]}')
                    return False
        return True
        
    
    def are_conditionaly_independant(self,name_Xs,name_Ys,name_Zs):
        Xs = [ X for X in list(self.Xs+self.Ys+self.Zs) if X.__qualname__ in name_Xs ]
        Ys = [ Y for Y in list(self.Xs+self.Ys+self.Zs) if Y.__qualname__ in name_Ys ]
        Zs = [ Z for Z in list(self.Xs+self.Ys+self.Zs) if Z.__qualname__ in name_Zs ]
        XYZ = RandomVariables(self.Omega,Xs=tuple(Xs+Ys),Ys=tuple(Zs))
        (zs_conditional_variables,zs_probabilities) = XYZ.distributions()
        for zs in zs_probabilities:
            are_Xs_Ys_independant_given_Zs_set_to_zs = \
                    zs_conditional_variables[zs].are_independant(name_Xs,name_Ys)
            #print(f'Zs={zs}  Xs and Ys independant ? {are_Xs_Ys_independant_given_Zs_set_to_zs}.')
            if not are_Xs_Ys_independant_given_Zs_set_to_zs:
                return False
        return True
    
    def list_all_possible_conditional_independances(self):
        # TODO
        pass
        
    def list_all_conditional_independances(self,debug=False):
        # TODO
        pass
    
    
    def print_distribution(self):
        d = self.distribution()
        names = [self.Xs[i].__qualname__ for i in range(len(self.Xs))]
        concatenated_names = ','.join(names)
        result = f'Distribution of ({concatenated_names})<:>\n'
        for xs in d:
            data = '('+','.join([ (f'{names[i]}={xs[i]}')for i in range(len(xs))])+')'
            result += f'P{data}={d[xs]}\n'
        result += '</:>'
        print(result)
        
    def print_distributions(self):
        (ys_conditional_variables,ys_probabilities) = self.distributions()
        print(f'ys conditional variables {list(ys_conditional_variables.keys())}')
        for ys in ys_conditional_variables:
            conditioning_evaluations = [f'{self.Ys[j].__qualname__}={ys[j]}' for j in range(len(self.Ys))]
            print('|('+','.join(conditioning_evaluations)+f') has probability {ys_probabilities[ys]}')
            ys_conditional_variables[ys].print_distribution()
            
    def print_variables(self):
        str_indices = ''
        str_variables = ''
        for i in range(len(self.Xs)):
            index_size = len(str(i))+1
            variable_size = len(self.Xs[i].__qualname__)
            if i < len(self.Xs)-1:
                variable_size += 1
            str_indices += f'{i}:'+max(variable_size-index_size,0)*' '
            str_variables += self.Xs[i].__qualname__+max(index_size-variable_size,0)*' '
            if i < len(self.Xs)-1:
                str_variables += ','
        str_indices += '|'
        str_variables += '|'
        for i in range(len(self.Ys)):
            index_size = len(str(len(self.Xs)+i))+1
            variable_size = len(self.Ys[i].__qualname__)
            if i < len(self.Ys)-1:
                variable_size += 1
            str_indices += f'{len(self.Xs)+i}:'+max(variable_size-index_size,0)*' '
            str_variables += self.Ys[i].__qualname__+max(index_size-variable_size,0)*' '
            if i < len(self.Ys)-1:
                str_variables += ','
        print(str_indices)
        print(str_variables)

In [23]:
## Check that unamed lambda functions fails:
X = RandomVariables(d12,Xs=lambda omega: omega%3)
print(X.Xs[0])
print(X.distribution())

AssertionError: 

TODO: Sur l'espace de probabilité d'un dé équilibré à 12 faces, créer la variable aléatoire donnant le résultat modulo 3 du dé. 

In [24]:
Rs = RandomVariables(d12,name('X',lambda omega: omega%2))
Rs.print_variables()
Rs.distribution()

0:|
X |


{(0,): Fraction(1, 2), (1,): Fraction(1, 2)}

TODO: Compléter la fonction suivante qui joint les variables aléatoires donnée en paramètre.

In [74]:
def join(variables:List[RandomVariables])->RandomVariables: #TODO
    ()

TODO: Définir la variable aléatoire $Y$ calculant la valeur modulo 4 d'un dé à d12 faces et tester l'indépendance de $X$ et $Y$ en utilisant une méthode de __RandomVariables__.

In [25]:
variables = [name('X',lambda omega: omega%3),name('Y',lambda omega: omega%4)]
RsXY = RandomVariables(d12,tuple(variables))
RsXY.print_variables()
RsXY.distribution()

0:1:|
X,Y |


{(0, 0): Fraction(1, 12),
 (1, 1): Fraction(1, 12),
 (2, 2): Fraction(1, 12),
 (0, 3): Fraction(1, 12),
 (1, 0): Fraction(1, 12),
 (2, 1): Fraction(1, 12),
 (0, 2): Fraction(1, 12),
 (1, 3): Fraction(1, 12),
 (2, 0): Fraction(1, 12),
 (0, 1): Fraction(1, 12),
 (1, 2): Fraction(1, 12),
 (2, 3): Fraction(1, 12)}

TODO: Conditionner selon la variable $X$ puis $Y$ et afficher les distributions $X|Y$ et $Y|X$.

In [77]:
def split(Xs:RandomVariables,selected_indices:List[int])->Tuple[RandomVariables]:
    number_of_decomposed_variables = len(list(Xs.distribution().keys())[0]) # XXX
    assert(max(selected_indices) < number_of_decomposed_variables)
    unselected_indices = [ i for i in range(number_of_decomposed_variables) if i not in selected_indices]
    selected_variables = tuple([ Xs.Xs[i] for i in selected_indices])
    unselected_variables = tuple([ Xs.Xs[i] for i in unselected_indices])
    return (RandomVariables(Xs.Omega,selected_variables),\
           RandomVariables(Xs.Omega,unselected_variables))

In [78]:
(Xr,Yr) = split(XY,[0])
print(Xr.distribution())
Xr.print_distribution()
Yr.print_distribution()

{(0,): Fraction(1, 4), (1,): Fraction(1, 4), (2,): Fraction(1, 4), (3,): Fraction(1, 4)}
Distribution of (Y)<:>
P(Y=0)=1/4
P(Y=1)=1/4
P(Y=2)=1/4
P(Y=3)=1/4
</:>
Distribution of (X)<:>
P(X=0)=1/3
P(X=1)=1/3
P(X=2)=1/3
</:>


In [79]:
toto = [1,2]
help(toto.insert)

Help on built-in function insert:

insert(...) method of builtins.list instance
    L.insert(index, object) -- insert object before index



### Conditionnement menant aux probabilités conditionnelles $P(X|Z)$ (et $P(X,Y|Z)$)

Soit $\Omega=\{1,\ldots 77\}$ l'espace de probabilité naturelement associé à un dé à $77$ faces équilibré munissant donc chaque évènement d'une probabilité uniforme $\frac{1}{77}$.

Considérons les variables aléatoires définient par 

$X(\omega) := \omega \mod 4$ en python lambda omega: omega % 4

et 

$Z(\omega) := \omega \mod 3$ en python lambda omega: omega % 2.

In [80]:
d7 = ProbabilitySpace(77)

Xd7 = RandomVariables(d7,name('Xd7',lambda omega: omega%4))
Zd7 = RandomVariables(d7,name('Zd7',lambda omega: omega%3))
XZd7 = join([Xd7,Zd7])
print(XZd7.distribution())
print(XZd7.are_independant(['Xd7'],['Zd7'],print_counter_example=True))
XZd7.print_distribution()
XZd7.print_variables()
XZd7.condition(['Zd7'])
XZd7.print_distributions()
XZd7.uncondition(['Zd7'])
XZd7.print_distributions()
XZd7.condition(['Xd7'])
XZd7.print_distributions()

{(0, 0): Fraction(1, 11), (1, 1): Fraction(1, 11), (2, 2): Fraction(1, 11), (3, 0): Fraction(1, 11), (0, 1): Fraction(1, 11), (1, 2): Fraction(6, 77), (2, 0): Fraction(6, 77), (3, 1): Fraction(6, 77), (0, 2): Fraction(6, 77), (1, 0): Fraction(6, 77), (2, 1): Fraction(6, 77), (3, 2): Fraction(6, 77)}
join((0, 0)) = 1/11,marginal_X[(0,)]=20/77, marginal_Y[(0,)]=26/77,marginal_X[(0,)]*marginal_Y[(0,)]=520/5929
False
Distribution of (Xd7,Zd7)<:>
P(Xd7=0,Zd7=0)=1/11
P(Xd7=1,Zd7=1)=1/11
P(Xd7=2,Zd7=2)=1/11
P(Xd7=3,Zd7=0)=1/11
P(Xd7=0,Zd7=1)=1/11
P(Xd7=1,Zd7=2)=6/77
P(Xd7=2,Zd7=0)=6/77
P(Xd7=3,Zd7=1)=6/77
P(Xd7=0,Zd7=2)=6/77
P(Xd7=1,Zd7=0)=6/77
P(Xd7=2,Zd7=1)=6/77
P(Xd7=3,Zd7=2)=6/77
</:>
0:  1: |
Xd7,Zd7|
ys conditional variables [(0,), (1,), (2,)]
|(Zd7=0) has probability 26/77
Distribution of (Xd7)<:>
P(Xd7=0)=7/26
P(Xd7=3)=7/26
P(Xd7=2)=3/13
P(Xd7=1)=3/13
</:>
|(Zd7=1) has probability 26/77
Distribution of (Xd7)<:>
P(Xd7=1)=7/26
P(Xd7=0)=7/26
P(Xd7=3)=3/13
P(Xd7=2)=3/13
</:>
|(Zd7=2) has 

### Une autre relation d'indépendance conditionnelle avec $\Omega$ espace de mots binaires.

Supposons que $\Omega$ soit l'ensemble des mots binaires de taille $5$, donc $\Omega := \{00000,00001,\ldots 11111\}$.
Pour en faire un espace probabilisé nous mettons une probabilité uniforme $\frac{1}{2^5}=\frac{1}{32}$ à chacun de ces mots.

Nous allons d'abord construire $3$ variables aléatoires $X',Y'$ et $Z'$ sur $\Omega$ qui seront évidemment indépendantes. Pour un mot $w:=\omega$ on utilise pour tout sous-ensemble de ses indices, la notation $w_I$ pour le sous-mot de $w=w_0w_1w_2w_3w_4$ formé des lettres aux indices de $I$ (et donc dans le même ordre). Nous utilisons aussi $\mathsf{bin}(w)$ pour la valeur en binaire de $w$.

$$ X'(w) = \mathsf{bin}(w_{\{0\}}), Y'(w) = \mathsf{bin}(w_{\{1,2,3\}}), Z'(w)=\mathsf{bin}(w_{\{4\}}).$$

Comme dans $\Omega$ les lettres $w_i$ sont indépendantes nous en déduisons que les variables $X',Y,'$ et $Z'$ le sont aussi

$$ P(X'=x',Y'=y',Z'=z') = P(X'=x')P(Y'=y')P(Z'=z').$$

Nous voyons donc que la lecture indépendante de lettres indépendantes conduit à des variables indépendantes.

Si nos considération sont valable pour toute réalisation de $Z'$ (ici $z'(w)=0$ aussi bien $z'(w)=1$), nous pouvons rassembler ces réalisations dans la notation $P(X',Y'|Z')$ sans discuter de la valeur de $z'(w)$.

Autrement dit, la notation $|Z'=z'(w)$ dans $P(X',Y'|Z'=z'(w))$ indique que parmi les triplets $(X'(w),Y'(w),Z'(w))$ nous ne retenons que les triplets où $Z'(w)=z'(w)$.

En python, dans une compréhension cela s'écrit [ (X'(w),Y'(w),Z'(w)) for w in Omega.all_omegas() if Z'(w) == k] avec $k=0$ ou $k=1$

Ici nous obtenons la partition des variables jointes, en deux, chacune définie par la valeur de $z'(w) = k$:

[ (X(w),Y(w), 1) for w in $1(0+1)^4$]

et 

[ (X(w),Y(w), 0) for w in $0(0+1)^4$].

induits l'indépendance de $X(w)$ et $Y(w)$, nous notons cette indépendance conditionnelle  sans discuter de la valeur de $Z$ par $P(X,Y |Z)$. 
    


Nous allons briser, _mais pas trop_, cette belle régularité qui jusque là permettait de factoriser les probabilités.

Nous allons définir les variables $X$, $Y$ et $Z$ de la façon suivante:

- $Z(w) = \mathsf{bin}(w_{\{0,\}})$ est le résultat d'une pièce non-biaisée qui va influencer la définition des variables $X$ et $Y$ car on utilise la notation $w=z(w)w'$ donc $w=0w'$ ou bien $w=1w'$.

- Si $Z(w)=0$: $X(0w') = \mathsf{bin}(w_{\{1\}})$ et $Y(0w') = \mathsf{bin}(w_{\{2,3,4\}})$.

- Si $Z(w)=1$: $X(1w') = \mathsf{bin}(w_{\{1,2,3\}})$ et $Y(1w')=\mathsf{bin}(w_{\{4\}})$.
    
Nous observons par les arguments précédents, que si $Z(w)$ est connu $X$ et $Y$ sont bien indépendants:

$$ P(X,Y|Z=z(w)) = P(X|Z=z(w))P(Y|Z=z(w))$$

où la notation $|Z=z(w)$ indique que parmi les triplets $(X(w),Y(w),Z(w))$ nous ne retenons que les triplets où $Z(w)=z(w)$.


 

TODO: Calculer l'entier en binaire associé par un tuple de bits.

In [5]:
def my_bin(w:Tuple[int])->int:
    return sum((2**i)*w[i] for i in range(len(w)))

In [6]:
print(my_bin((1,0,1)))
print(my_bin(()))


5
0


TODO: Définir le générateur d'espaces probabiliste donnant la mesure uniforme sur tout les tuples binaire de taille $k$:

In [54]:
def generate_OmegaBin_k(k:int)->ProbabilitySpace:
    mes={bin:1/(2**k) for bin in itertools.product(range(2),repeat=k)}
    space = ProbabilitySpace(nb_events=len(mes),measure=mes)
    return space

In [55]:
OmegaBin_5 = generate_OmegaBin_k(5)
print(OmegaBin_5.measure)
for w in OmegaBin_5.all_omegas():
    print(w)

{(0, 0, 0, 0, 0): 0.03125, (0, 0, 0, 0, 1): 0.03125, (0, 0, 0, 1, 0): 0.03125, (0, 0, 0, 1, 1): 0.03125, (0, 0, 1, 0, 0): 0.03125, (0, 0, 1, 0, 1): 0.03125, (0, 0, 1, 1, 0): 0.03125, (0, 0, 1, 1, 1): 0.03125, (0, 1, 0, 0, 0): 0.03125, (0, 1, 0, 0, 1): 0.03125, (0, 1, 0, 1, 0): 0.03125, (0, 1, 0, 1, 1): 0.03125, (0, 1, 1, 0, 0): 0.03125, (0, 1, 1, 0, 1): 0.03125, (0, 1, 1, 1, 0): 0.03125, (0, 1, 1, 1, 1): 0.03125, (1, 0, 0, 0, 0): 0.03125, (1, 0, 0, 0, 1): 0.03125, (1, 0, 0, 1, 0): 0.03125, (1, 0, 0, 1, 1): 0.03125, (1, 0, 1, 0, 0): 0.03125, (1, 0, 1, 0, 1): 0.03125, (1, 0, 1, 1, 0): 0.03125, (1, 0, 1, 1, 1): 0.03125, (1, 1, 0, 0, 0): 0.03125, (1, 1, 0, 0, 1): 0.03125, (1, 1, 0, 1, 0): 0.03125, (1, 1, 0, 1, 1): 0.03125, (1, 1, 1, 0, 0): 0.03125, (1, 1, 1, 0, 1): 0.03125, (1, 1, 1, 1, 0): 0.03125, (1, 1, 1, 1, 1): 0.03125}
(0, 0, 0, 0, 0)
(0, 0, 0, 0, 1)
(0, 0, 0, 1, 0)
(0, 0, 0, 1, 1)
(0, 0, 1, 0, 0)
(0, 0, 1, 0, 1)
(0, 0, 1, 1, 0)
(0, 0, 1, 1, 1)
(0, 1, 0, 0, 0)
(0, 1, 0, 0, 1)
(0, 1, 

TODO: Définir les variables $X$, $Y$ et $Z$ sur les mots binaires de 5 bits vue précédemment.

In [56]:
Bin_5_variables = [\
        name('X',lambda w: bin(w[1:4]) if w[0]==0 else bin(w[1:2])),\
        name('Y',lambda w: bin(w[4:5]) if w[0]==0 else bin(w[2:5])),\
        name('Z',lambda w: bin(w[0:1])),\
    ]

RBin_5 = RandomVariables(OmegaBin_5,tuple(Bin_5_variables))

In [57]:
RBin_5.print_distributions()
are_XY_independant = RBin_5.are_independant(['X'],['Y'])
print(f'are X,Y independant {are_XY_independant} ')

ys conditional variables [()]
|() has probability 1.0


TypeError: 'tuple' object cannot be interpreted as an integer

### Toutes les relations d'indépendance conditionnelle

Pour un ensemble de $k$ variables aléatoires $X_1,\ldots X_k$ sur le même espace de probabilité $\Omega$, on se donne: $X$, $Y$ et $Z$ trois sous-ensembles disjoints de variables tels que $X$ et $Y$ soient non-vide.

Pour chaque triplet, nous voulons savoir si $X$ et $Y$ sont indépendants sachant $Z$. 

Autrement dit, dans une approximation intuitive que nous revisiterons plus tard, l'ensemble $Z$ contient tout ce qui pourrait être commun à la définition de $X$ et $Y$ et rien de ce qui pourrait être déduit à la fois des réalisations $X$ et $Y$.

Cela se donne $I(X,Y,Z) := X\perp\!\!\!\perp Y | Z$
et se nomme l'__indépendance de $X$ et $Y$ conditionellement à $Z$__. 

Par définition 
$$ X\perp\!\!\!\perp Y | Z \iff \forall z \left(,\exists \omega\in \Omega,z=Z(\omega) \implies \right) P(X,Y|Z=z)=P(X|Z=z)P(Y|Z=z).$$

C'est cette définition que nous pouvons tester avec __RandomVariables.are_conditionaly_independant__ sur tous les triplets de variables qu'il est possible d'énumérer avec __RandomVariables.list_all_possible_conditional_independances__ 

In [58]:
d60 = ProbabilitySpace(60)

Xs = [\
      name('X',lambda omega: omega % 2),\
      name('Y',lambda omega: (3*omega+5)%7),\
      name('Z',lambda omega: 2*omega%9),\
      name('T',lambda omega: omega**2%13),\
     ]

XYZT = RandomVariables(d60,tuple(Xs))

XYZT.print_distributions()



ys conditional variables [()]
|() has probability 1
Distribution of (X,Y,Z,T)<:>
P(X=0,Y=5,Z=0,T=0)=1/60
P(X=1,Y=1,Z=2,T=1)=1/60
P(X=0,Y=4,Z=4,T=4)=1/60
P(X=1,Y=0,Z=6,T=9)=1/60
P(X=0,Y=3,Z=8,T=3)=1/60
P(X=1,Y=6,Z=1,T=12)=1/60
P(X=0,Y=2,Z=3,T=10)=1/60
P(X=1,Y=5,Z=5,T=10)=1/60
P(X=0,Y=1,Z=7,T=12)=1/60
P(X=1,Y=4,Z=0,T=3)=1/60
P(X=0,Y=0,Z=2,T=9)=1/60
P(X=1,Y=3,Z=4,T=4)=1/60
P(X=0,Y=6,Z=6,T=1)=1/60
P(X=1,Y=2,Z=8,T=0)=1/60
P(X=0,Y=5,Z=1,T=1)=1/60
P(X=1,Y=1,Z=3,T=4)=1/60
P(X=0,Y=4,Z=5,T=9)=1/60
P(X=1,Y=0,Z=7,T=3)=1/60
P(X=0,Y=3,Z=0,T=12)=1/60
P(X=1,Y=6,Z=2,T=10)=1/60
P(X=0,Y=2,Z=4,T=10)=1/60
P(X=1,Y=5,Z=6,T=12)=1/60
P(X=0,Y=1,Z=8,T=3)=1/60
P(X=1,Y=4,Z=1,T=9)=1/60
P(X=0,Y=0,Z=3,T=4)=1/60
P(X=1,Y=3,Z=5,T=1)=1/60
P(X=0,Y=6,Z=7,T=0)=1/60
P(X=1,Y=2,Z=0,T=1)=1/60
P(X=0,Y=5,Z=2,T=4)=1/60
P(X=1,Y=1,Z=4,T=9)=1/60
P(X=0,Y=4,Z=6,T=3)=1/60
P(X=1,Y=0,Z=8,T=12)=1/60
P(X=0,Y=3,Z=1,T=10)=1/60
P(X=1,Y=6,Z=3,T=10)=1/60
P(X=0,Y=2,Z=5,T=12)=1/60
P(X=1,Y=5,Z=7,T=3)=1/60
P(X=0,Y=1,Z=0,T=9)=1/60
P(X=1,Y=4,Z=2,T=4)=

L'exemple "aléatoire à la main" étudier nous suggère donc d'étudier la distribution $TX|YZ$ qui montre pour chaque $Y$ et $Z$ fixés l'indépendance de $T$ et $X$.

In [59]:
XYZT.condition(['Z','Y'])
XYZT.print_variables()
XYZT.print_distributions()

0:1:|2:3:
X,T |Y,Z 
ys conditional variables [(5, 0), (1, 2), (4, 4), (0, 6), (3, 8), (6, 1), (2, 3), (5, 5), (1, 7), (4, 0), (0, 2), (3, 4), (6, 6), (2, 8), (5, 1), (1, 3), (4, 5), (0, 7), (3, 0), (6, 2), (2, 4), (5, 6), (1, 8), (4, 1), (0, 3), (3, 5), (6, 7), (2, 0), (5, 2), (1, 4), (4, 6), (0, 8), (3, 1), (6, 3), (2, 5), (5, 7), (1, 0), (4, 2), (0, 4), (3, 6), (6, 8), (2, 1), (5, 3), (1, 5), (4, 7), (0, 0), (3, 2), (6, 4), (2, 6), (5, 8), (1, 1), (4, 3), (0, 5), (3, 7), (6, 0), (2, 2), (5, 4), (1, 6), (4, 8), (0, 1)]
|(Y=5,Z=0) has probability 1/60
Distribution of (X,T)<:>
P(X=0,T=0)=1
</:>
|(Y=1,Z=2) has probability 1/60
Distribution of (X,T)<:>
P(X=1,T=1)=1
</:>
|(Y=4,Z=4) has probability 1/60
Distribution of (X,T)<:>
P(X=0,T=4)=1
</:>
|(Y=0,Z=6) has probability 1/60
Distribution of (X,T)<:>
P(X=1,T=9)=1
</:>
|(Y=3,Z=8) has probability 1/60
Distribution of (X,T)<:>
P(X=0,T=3)=1
</:>
|(Y=6,Z=1) has probability 1/60
Distribution of (X,T)<:>
P(X=1,T=12)=1
</:>
|(Y=2,Z=3) has probabil

Et sans trop de surprise, nous constatons que cette indépendance conditionnelle provient du fait que la connaissance de $Y$ et $Z$ détermine $\omega$ et donc que les lois conditionnelle sont indépendantes car finalement réduite à un évènement atomique.

L'écriture de cette recension des relations d'indépendance conditionnelle nous permets de compléter la méthode __RandomVariables.list_all_conditional_independances__.

Nous allons considérer des indépendances un peu plus structurer sur l'exemple suivant (toujours sur $\Omega=$__d60__) pour trouver des indépendances conditionnelles plus riches.

In [60]:
Xs = [\
    name('X',lambda omega: omega % 2),
    name('Y',lambda omega: omega % 3),
    name('Z',lambda omega: (omega+3) % 6),
    name('T',lambda omega: omega % 7),
    name('U',lambda omega: (2*omega+7) % 15),
]

R = RandomVariables(d60,tuple(Xs))
triplets = R.list_all_conditional_independances(debug=True)
print(f'We found {len(triplets)} conditional independances')

TypeError: object of type 'NoneType' has no len()